In [48]:
import numpy as np
import pandas as pd
import sys
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.metrics
import sklearn
import lazypredict
import matplotlib.pyplot as plt
from tqdm.notebook import tnrange
import itertools
from sklearn.impute import KNNImputer
import tqdm
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import svm
import optuna
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
from sklearn.preprocessing import FunctionTransformer
from fitter import Fitter, get_common_distributions, get_distributions
import time
from datetime import datetime
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [71]:
data_files = {
    "train_airbnb": "../DATA/train_airbnb_berlin.xls",
    "berlin_crimes": "../DATA/extra_datasets/Berlin_crimes.csv",
    "berlin_demographics": "../DATA/extra_datasets/Berlin_demographics.csv"
}

def cast_district(x):
    return x.replace("NeukÃ¶lln","Neukölln").replace('Tempelhof - SchÃ¶neberg','Tempelhof - Schöneberg').replace('Treptow - KÃ¶penick','Treptow - Köpenick').replace('Charlottenburg-Wilm.','Charlottenburg-Wilmersdorf').replace('Ã¶','ö').replace('Ã¤','ä').replace('Ã','ß')

def get_training_data(data_files):
    df_train = pd.read_csv(data_files['train_airbnb'])

    #crimes_df = pd.read_csv(data_files['berlin_crimes'])
    #crimes_df = crimes_df.groupby(['District','Year']).sum().groupby('District').mean().drop('Code', axis=1)

    #demographics_df = pd.read_csv(data_files['berlin_demographics'], delimiter=';', encoding='latin-1')
    #demographics_df['Code Postal'] = demographics_df.apply(lambda x: x['Bezirk']*10000+x['Ortsteil']*10+x['Geschl'], axis=1)
    #demographics_df = demographics_df[['Code Postal', 'Bez-Name', 'Ortst-Name', 'Staatsangeh','Altersgr', 'Häufigkeit']]
    #demographics_df = demographics_df.rename(columns = {"Bez-Name":"District","Häufigkeit": "Population", "Altersgr":"Tranche d'âge","Staatsangeh":"Origine"})
    #number_of_inhabitants_df = demographics_df.groupby(['Code Postal','District','Ortst-Name']).sum()
    #number_of_germans_df = demographics_df.groupby(['Code Postal','District','Ortst-Name','Origine']).sum().query('Origine=="D"').rename(columns = {"Population":"Nb Allemands"})
    #number_of_immigrants_df = demographics_df.groupby(['Code Postal','District','Ortst-Name','Origine']).sum().query('Origine=="A"').rename(columns = {"Population":"Nb Immigrants"})
    #demographics_df = number_of_inhabitants_df.merge(number_of_germans_df, how='inner', on=['Code Postal','District','Ortst-Name']).merge(number_of_immigrants_df, how='inner', on=['Code Postal','District','Ortst-Name'])
    #demographics_df['Proportion Allemands'] = demographics_df.apply(lambda x: x['Nb Allemands']/x['Population'], axis=1)
    #demographics_df['Proportion Immigrants'] = demographics_df.apply(lambda x: x['Nb Immigrants']/x['Population'], axis=1)
    #demographics_df = demographics_df[['Population','Proportion Allemands','Proportion Immigrants']]

    #extra_data_df = demographics_df.reset_index().merge(crimes_df, how='left', on='District')
    #for column in extra_data_df.columns:
        #if column in ['Robbery','Street_robbery', 'Injury', 'Agg_assault', 'Threat', 'Theft', 'Car','From_car', 'Bike', 'Burglary', 'Fire', 'Arson', 'Damage', 'Graffiti','Drugs', 'Local']:
            #extra_data_df[column] = extra_data_df.apply(lambda x: x[column]/x['Population'], axis=1)
    
    #df_train['neighbourhood'] = df_train['neighbourhood'].apply(lambda x:cast_district(x))
    #df_train['Neighborhood Group'] = df_train['Neighborhood Group'].apply(lambda x:cast_district(x))
    #df_train = df_train.merge(extra_data_df, how='left', left_on='neighbourhood', right_on='Ortst-Name')

    return df_train

In [72]:
df_train = get_training_data(data_files)

# Pre processing

### Feature analysis

In [73]:
def get_subplot_analysis():
    plt.subplot(221)
    df_train.boxplot(column='Price')

    plt.subplot(222)
    df_train.boxplot(column='Square Feet')

    plt.subplot(223)
    df_train.boxplot(column='Value Rating')

    plt.subplot(224)
    df_train.boxplot(column='Accuracy Rating')

    plt.show()

In [74]:
def percentage_of_nans(df: pd.DataFrame) -> list:
    nan_percentage = pd.DataFrame(columns=['Feature Name', 'Percentage of NaNs'])
    for idx, feature in enumerate(df.columns):
        notnans = df_train[feature].notna().value_counts()
        percentage = 0
        if False in notnans:
            percentage = int(10000*notnans[False]/len(df_train))/100

        nan_percentage.loc[idx] = [feature, percentage]
    return nan_percentage

In [75]:
def nan_percentage_analysis():
    nan_percentage = percentage_of_nans(df_train)
    df = nan_percentage[nan_percentage['Percentage of NaNs'] >= 18]
    df.sort_values(by=['Percentage of NaNs'], ascending=False).reset_index().drop(['index'], axis=1)

### Data cleaning

In [81]:
def pre_process_raw(df: pd.DataFrame):
    df.columns = df.columns. str.lower().str.replace(' ','_')         

    # Converting to datetime
    df.host_since = pd.to_datetime(df.host_since)               

    cols_to_drop = ['listing_name', 'host_id', 'host_name', 'city', 'country_code', 'country', 
                    'first_review', 'last_review', 'neighbourhood', 'business_travel_ready',  'postal_code']
    df = df.drop(cols_to_drop, axis=1)
    df.set_index('listing_id', inplace=True)

    df.drop(['square_feet'], axis=1, inplace=True)

    for col in df.columns:
        df[col] = df[col].replace('*', np.nan)

    df = df.dropna(subset=['price'], how='all')
    df = df.dropna(subset=['overall_rating', 'accuracy_rating','cleanliness_rating','checkin_rating',
                            'communication_rating','location_rating','value_rating'], how='all')
    
    change = ['accomodates','latitude',
          'longitude','bathrooms','bedrooms','beds','guests_included','min_nights','reviews','overall_rating','accuracy_rating',
          'cleanliness_rating','checkin_rating', 'communication_rating','location_rating','value_rating']

    label_encoded = ['host_response_time', 'is_superhost', 'is_exact_location', 'instant_bookable']

    categorical_variables = ['room_type', 'property_type','neighborhood_group'] 
    for i in change:
        df[i]=pd.to_numeric(df[i], downcast="float")

    #Replacing the null values

    df['host_response_rate'] = df['host_response_rate'].map(lambda x: str(x), na_action='ignore').map(lambda x : int(x[:-1]), na_action='ignore') 

    for cat_var in categorical_variables:
        dummie = pd.get_dummies(df[cat_var])
        df = pd.concat([df, dummie], axis=1)

    df.drop(categorical_variables, axis=1, inplace=True)
    
    le = LabelEncoder()
    for i in label_encoded:
        df[i] = le.fit_transform(df[i].astype(str))
    
    X = df.drop('price', axis = 1)

    # Output/Dependent variable

    y = df['price']
    return X, y

def median_imputer(df: pd.DataFrame):
    change = ['accomodates','latitude',
        'longitude','bathrooms','bedrooms','beds','guests_included','min_nights','reviews','overall_rating','accuracy_rating',
        'cleanliness_rating','checkin_rating', 'communication_rating','location_rating','value_rating' ]
    
    df = df.copy()

    if 'host_since' in df.columns:
        # Calculating the number of days    
        df['host_days_active'] = (datetime(2019, 7, 21) - df.host_since).astype('timedelta64[D]').copy()

        # Replacing null values with the median
        df['host_days_active'] = df.host_days_active.fillna(df.host_days_active.median()).copy()

        df['host_days_active'] = (datetime(2019, 7, 21) - df.host_since).astype('timedelta64[D]').copy()
        df.drop('host_since', axis=1, inplace=True)

    for col in change:
        if col not in df.columns:continue
        df[col].fillna(df[col].median(), inplace=True)

    df = df.fillna(0).copy()

    return df

In [82]:
def fit_clf_scaler_cv(clf, X, y, clf_params, pipeline_params={}, n_features=50):
    X = X.copy()
    y = y.copy()
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    kf = KFold(n_splits=5)
    
    rmse_list = []
    r2_list = []
    X = median_imputer(X)

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]

        pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=n_features)),
        ('svr', clf(**clf_params))], **pipeline_params)
        
        pipe.fit(X_train,y_train)
        y_pred = pipe.predict(X_test)
        rmse = mean_squared_error(y_pred, y_test)**(1/2)
        rmse_list.append(rmse)
        r2_list.append(pipe.score(X_test, y_test))
    return np.array(rmse_list).mean(), np.array(r2_list).mean()

def fit_clf_scaler(clf, X, y, clf_params, pipeline_params={}, n_features=50):
    X = X.copy()
    y = y.copy()
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    
    pipe = Pipeline([('median_imputer', FunctionTransformer(median_imputer)),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=n_features)),
    ('svr', clf(**clf_params))], **pipeline_params)
    
    pipe.fit(X,y)
    return pipe 

In [83]:
X, y = pre_process_raw(df_train)

In [84]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

### Train test split

In [85]:
def get_train_test_analysis():
    plt.hist(y_train,bins=100, label='train')
    plt.hist(y_test, bins=100,label='test')
    plt.legend()
    plt.grid(axis='x')
    plt.show()

    f = Fitter(y.values,
            distributions=['gamma',
                            'lognorm',
                            "beta",
                            "burr",
                            "norm"])
    f.fit()
    f.summary()

    f_train = Fitter(y_train.values, distributions=['burr'])
    f_train.fit()

    display(f_train.get_best())

    f_test = Fitter(y_test.values, distributions=['burr'])
    f_test.fit()
    display(f_test.get_best())

    r_train = burr.rvs(**(f_train.get_best()['burr']),size=1000)
    r_test = burr.rvs(**(f_test.get_best()['burr']),size=1000)
    r_test_base = burr.rvs(**(f_test.get_best()['burr']),size=1000)
    print('KL divergence between train and test: ', sum(rel_entr(r_train, r_test)))
    print('KL divergence reference: ', sum(rel_entr(r_test, r_test_base)))

# Feature analysis

### PCA

In [94]:
def make_pca_analysis():
    pipe = Pipeline([('median_imputer', FunctionTransformer(median_imputer)),
    ('scaler', StandardScaler()),
    ('pca', PCA())])

    pipe.fit(X_train)
    explained_variance = pipe['pca'].explained_variance_ratio_
    acc_sum = np.cumsum(explained_variance) 

    plt.title('Cumulative explained variance')
    plt.plot(acc_sum)
    plt.grid()
    plt.ylabel('Explained variance')
    plt.xlabel('Number of features')
    plt.show()

    print('Explained variance with 50 features: ', acc_sum[45])

### Forward selection + Cross Validation

In [13]:
def get_feature_importance(clf, params):
	r2 = [] 
	rmse = []
	features = []
	n_features = []

	for k in range(1,len(X.columns) + 1):
		r2_tmp = []
		rmse_tmp = []
		for new_feat in X_train.columns:
			if new_feat in features: continue
			new_features = features.copy()
			new_features.append(new_feat)
			pipe = fit_clf_scaler(clf, X_train[new_features],y_train, params, {}, n_features=len(new_features))   
			r_squared = pipe.score(X_test[new_features],y_test)
			y_pred = pipe.predict(X_test[new_features])
			rmse_val = mean_squared_error(y_pred, y_test)**(1/2)
			r2_tmp.append((r_squared, new_feat))                 
			rmse_tmp.append(rmse_val)
		r2.append(sorted(r2_tmp, key=lambda x: x[0])[-1][0])	
		rmse.append(sorted(rmse_tmp)[-1])
		new_feat = sorted(r2_tmp, key=lambda x: x[0])[-1][1]
		features.append(new_feat)
		n_features.append(len(features))   

	df = pd.DataFrame({'n_features': n_features,'R2': r2, 'RMSE':rmse ,'features':features})

	return df

In [14]:
def get_subset_selection_analysis():
    df = get_feature_importance(linear_model.LinearRegression, {})
    plt.title('Cross-validated subset selection - Linear Regression')
    plt.xlabel('Number of features')
    plt.ylabel('R squared')
    plt.grid()
    plt.plot(df['n_features'], df['R2'])
    plt.show()

    plt.title('Cross-validated subset selection - Linear Regression')
    plt.xlabel('Number of features')
    plt.ylabel('RMSE')
    plt.grid()
    plt.plot(df['n_features'], df['RMSE'])
    plt.show()


# Model selection

### Decision Trees

#### Analysis

In [15]:
def get_decision_tree_analysis():
    rmse = []
    max_depth = []
    for k in range(1, 30):
        rmse_val = fit_clf_scaler_cv(DecisionTreeRegressor, X_train,y_train, {"max_depth": k}, {"verbose": 0}, n_features=25) 
        rmse.append(rmse_val)	
        max_depth.append(k)

    df = pd.DataFrame({'RMSE': rmse,'max_depth':max_depth})
    plt.title('Influence of the max depth - Decision Tree Regression - 5 fold CV')
    plt.xlabel('Max depth')
    plt.ylabel('RMSE')
    plt.grid()
    plt.plot(df['max_depth'], df['RMSE'])
    plt.show()

#### Model selection

In [96]:
clf_params = {'max_depth': 4, 'min_samples_split': 2}
rmse, r_squared = fit_clf_scaler_cv(DecisionTreeRegressor, X_train, y_train, clf_params)

print('r_squared: ', r_squared)
print('rmse: ', rmse)

r_squared:  0.31403471430716545
rmse:  40.01658608039282


### SVM

#### Optimization

In [17]:
def optimize_smv():
    def objective(trial: optuna.trial.Trial):

        parameters = {
            'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid']),
            'C': trial.suggest_float('C', 1e-3, 1e3),
            'epsilon': trial.suggest_float('epsilon', 1e-3, 1)
        }

        clf = svm.SVR
        return -fit_clf_scaler(clf, X_train, y_train, parameters)

    study = optuna.create_study()
    study.optimize(objective, n_trials=1)

#### Model selection

In [97]:
clf_params = {'kernel': 'rbf'}#, 'C': 595.6756199722245, 'epsilon': 0.8771538699650302}
rmse, r_squared = fit_clf_scaler_cv(svm.SVR, X_train, y_train, clf_params, {"verbose": 0})

print('r_squared: ', r_squared)
print('rmse: ', rmse)

r_squared:  0.27329516235459844
rmse:  41.24319778781084


In [33]:
clf_params = {'kernel': 'rbf'}#, 'C': 595.6756199722245, 'epsilon': 0.8771538699650302}
pipe = fit_clf_scaler(svm.SVR, X_train, y_train, clf_params, {"verbose": 1})
r_squared = pipe.score(X_test,y_test)
y_pred = pipe.predict(X_test)
rmse = mean_squared_error(y_pred, y_test)**(1/2)

print('r_squared: ', r_squared)
print('rmse: ', rmse)

[Pipeline] .... (step 1 of 4) Processing median_imputer, total=   0.0s
[Pipeline] ............ (step 2 of 4) Processing scaler, total=   0.0s
[Pipeline] ............... (step 3 of 4) Processing pca, total=   0.9s
[Pipeline] ............... (step 4 of 4) Processing svr, total=  21.4s
r_squared:  0.33655831510958
rmse:  39.16692711061652


### AdaBoost

#### Optimization

In [34]:
def optimize_adaboost():
    def objective(trial: optuna.trial.Trial):

        parameters = {
            'n_estimators': trial.suggest_int('n_estimators', 1, 200),
            'learning_rate': trial.suggest_float('learning_rate', 1e-6, 1)
        }

        clf = AdaBoostRegressor
        return -fit_clf_scaler(clf, X_train, y_train, parameters, n_features=5)

    study = optuna.create_study()
    study.optimize(objective, n_trials=10)

#### Model selection

In [98]:
clf_params = {'n_estimators': 119, 'learning_rate': 0.9617041908836014}
rmse, r_squared = fit_clf_scaler_cv(AdaBoostRegressor, X_train, y_train, clf_params, {"verbose": 0})

print('r_squared: ', r_squared)
print('rmse: ', rmse)

r_squared:  -0.8278628931220486
rmse:  64.60127861678401


### KNN

#### Optimization

In [36]:
def optimize_knn():
    def objective(trial: optuna.trial.Trial):

        trial_suggestion = {
            'n_neighbors': trial.suggest_int('n_neighbors', 1, 200),
            'leaf_size': trial.suggest_int('leaf_size', 10, 50),
        }

        clf_params = {
            'n_neighbors': trial_suggestion['n_neighbors'],
            'leaf_size': trial_suggestion['leaf_size'],
            'n_jobs': -1
        }
        imputer_params = {
            "n_neighbors": 3
        }

        clf = KNeighborsRegressor
        return fit_clf_scaler_cv(clf, X_train, y_train, clf_params, imputer_params, n_features=28)

    study = optuna.create_study()
    study.optimize(objective, n_trials=30, n_jobs=-1)

#### Model Selection

In [99]:
clf_params =  {'n_neighbors': 84, 'leaf_size': 26}
rmse, r_squared = fit_clf_scaler_cv(KNeighborsRegressor, X_train, y_train, clf_params, {"verbose": 0})
print('r_squared: ', r_squared)
print('rmse: ', rmse)

r_squared:  0.38255890047340396
rmse:  38.00313486098797


### Gradient Tree Boosting

#### Optimization

In [ ]:
def optimize_gradient_boost():
    def objective(trial: optuna.trial.Trial):

        trial_suggestion = {
            'n_estimators': trial.suggest_int('n_estimators', 1, 200),
            'learning_rate': trial.suggest_float('learning_rate', 1e-6, 1),
            'max_depth': trial.suggest_int('max_depth', 1, 20),
            'n_neighbors': trial.suggest_int('n_neighbors',  2, 4),
            'n_features': trial.suggest_int('n_features', 10, 54)
        }

        clf_params = {
            'n_estimators': trial_suggestion['n_estimators'],
            'learning_rate': trial_suggestion['learning_rate'],
            'max_depth': trial_suggestion['max_depth'],
        }
        imputer_params = {
            "n_neighbors": trial_suggestion['n_neighbors']
        }

        clf = GradientBoostingRegressor
        return fit_clf_scaler_cv(clf, X_train, y_train, clf_params, imputer_params, n_features=trial_suggestion['n_features'])

    study = optuna.create_study()
    study.optimize(objective, n_trials=10, n_jobs=-1)

#### Model selection

In [100]:
clf_params = {'n_estimators': 178, 'learning_rate': 0.15253399946567545, 'max_depth': 2} 
rmse, r_squared = fit_clf_scaler_cv(GradientBoostingRegressor, X_train, y_train, clf_params, {"verbose": 0})

print('r_squared: ', r_squared)
print('rmse: ', rmse)

r_squared:  0.43583816434933464
rmse:  36.335879959122224


### MLP

#### Optimization

In [ ]:
simplefilter("ignore", category=ConvergenceWarning)
def optimize_mlp():
    def objective(trial: optuna.trial.Trial):

        trial_suggestions = {
            'first_layer': trial.suggest_int('first_layer', 1, 100),
            'second_layer': trial.suggest_int('second_layer', 1, 100),
            'activation': trial.suggest_categorical('activation', ['relu', 'logistic', 'tanh', 'relu'])
        }

        clf = MLPRegressor 
        clf_params = {
            'hidden_layer_sizes': (trial_suggestions['first_layer'], trial_suggestions['second_layer']),#, trial_suggestions['third_layer']),
            'activation': trial_suggestions['activation'],
            'max_iter': 200,
            'learning_rate': 'adaptive'
        }
        rmse, r_squared = fit_clf_scaler_cv(clf, X_train, y_train, clf_params, {"verbose": 0})
        return rmse

    study = optuna.create_study()
    study.optimize(objective, n_trials=4)

#### Model selection

In [101]:
clf_params = {
    "hidden_layer_sizes": (93, 47),
    "activation": 'tanh'
}
rmse, r_squared = fit_clf_scaler_cv(sklearn.neural_network.MLPRegressor , X_train, y_train, clf_params, {"verbose": 0})

print('r_squared: ', r_squared)
print('rmse: ', rmse)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


r_squared:  0.30530548635178556
rmse:  40.30930851552766


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [102]:
clf_params = {
    "hidden_layer_sizes": (93, 47),
    "activation": 'logistic'
}
pipe = fit_clf_scaler(sklearn.neural_network.MLPRegressor, X_train, y_train, clf_params, {"verbose": 1})
r_squared = pipe.score(X_test,y_test)
y_pred = pipe.predict(X_test)
rmse = mean_squared_error(y_pred, y_test)**(1/2)

print('r_squared: ', r_squared)
print('rmse: ', rmse)

[Pipeline] .... (step 1 of 4) Processing median_imputer, total=   0.0s
[Pipeline] ............ (step 2 of 4) Processing scaler, total=   0.0s
[Pipeline] ............... (step 3 of 4) Processing pca, total=   0.0s
[Pipeline] ............... (step 4 of 4) Processing svr, total=  40.7s
r_squared:  0.39140806252137494
rmse:  33.35246856130338


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### XGBoost

#### Model selection

In [103]:
from xgboost import XGBRegressor
clf_params = {
    "learning_rate": 0.001,
    "n_estimators":6000,
    "max_depth":4,
    "min_child_weight":0,
    "gamma":0.6,
    "subsample":0.7,
    "colsample_bytree":0.7,
    "objective":'reg:squarederror',
    "nthread":-1,
    "scale_pos_weight":1,
    "seed":27,
    "reg_alpha":0.00006,
    "random_state":42
}
rmse, r_squared = fit_clf_scaler_cv(XGBRegressor, X_train, y_train, clf_params, {"verbose": 0})

print('r_squared: ', r_squared)
print('rmse: ', rmse)

### Random Forest

#### Optimization

In [ ]:
def optimize_random_forest():
    def objective(trial: optuna.trial.Trial):

        trial_suggestion = {
            'n_estimators': trial.suggest_int('n_estimators', 1, 200),
            'max_depth': trial.suggest_int('max_depth', 1, 20),
            'n_neighbors': trial.suggest_int('n_neighbors',  2, 4),
        }

        clf_params = {
            "n_estimators": trial_suggestion['n_estimators'],
            "criterion": "squared_error",

        }
        imputer_params = {
            "n_neighbors": trial_suggestion['n_neighbors']
        }

        clf = GradientBoostingRegressor
        return fit_clf_scaler_cv(clf, X_train, y_train, clf_params, imputer_params, n_features=28)

    study = optuna.create_study()
    study.optimize(objective, n_trials=10, n_jobs=-1)

#### Model selection

In [ ]:
clf_params = {'n_estimators': 134, 'max_depth': 5}
rmse, r_squared = fit_clf_scaler_cv(RandomForestRegressor, X_train, y_train, clf_params, {"verbose": 0})

print('r_squared: ', r_squared)
print('rmse: ', rmse)

r_squared:  0.40183903267545507
rmse:  36.441424499650196
